# 1. 아이허브 웹크롤링

In [5]:
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from selenium import webdriver   # 웹 브라우저 자동화
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm_notebook   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [21]:
skincare=[1,2,3,4,5,6]
mask=[37,38,39,40]
suncare =[41,42,43,44,45,46,47]
keywords= skincare + mask+ suncare
keywords

[1, 2, 3, 4, 5, 6, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]

In [22]:
# Step 1. 크롬 웹브라우저 실행

# pip install chromedriver_autoinstaller

# glowpick 스킨카테고리
for keyword in keywords:
    driver = webdriver.Chrome("chromedriver.exe")
    driver.get('https://glowpick.com/categories/{}'.format(i))
    time.sleep(2)  # 2초간 정지

# 제품 url들 수집
url_list = []
name_list = []
num_list =[]

# URL_raw 크롤링 시작
items = ".absolute-link-wrapper > a"
item_raw = driver.find_elements_by_css_selector(items)



for i in (0,24):
# url 크롤링    
    url = item_raw[i].get_attribute('href')
    url_list.append(url)

# 상품번호 크롤링
    num = item_raw[i].get_attribute('data-product-id')
    num_list.append(num)
# 제목 크롤링 
    name = item_raw[i].get_attribute('title')
    name_list.append(name)

    print(name)
    time.sleep(2)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=98.0.4758.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x0083A0A3+2597027]
	Ordinal0 [0x007CBA61+2144865]
	Ordinal0 [0x006C3B20+1063712]
	Ordinal0 [0x006B8242+1016386]
	Ordinal0 [0x006B8A78+1018488]
	Ordinal0 [0x006BA2B2+1024690]
	Ordinal0 [0x006B3F99+999321]
	Ordinal0 [0x006C4F90+1068944]
	Ordinal0 [0x007182E2+1409762]
	Ordinal0 [0x007085C6+1344966]
	Ordinal0 [0x006E54C6+1201350]
	Ordinal0 [0x006E63B6+1205174]
	GetHandleVerifier [0x009E10A2+1681842]
	GetHandleVerifier [0x00A93B0C+2413596]
	GetHandleVerifier [0x008D1221+568113]
	GetHandleVerifier [0x008CFFA3+563379]
	Ordinal0 [0x007D174E+2168654]
	Ordinal0 [0x007D6538+2188600]
	Ordinal0 [0x007D6680+2188928]
	Ordinal0 [0x007E030C+2229004]
	BaseThreadInitThunk [0x76FCFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77CB7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77CB7A6E+238]


In [49]:
df = pd.DataFrame({'url':url_list, 'name':name_list, 'item_id':num_list})
# 저장하기
df.to_excel("iherb_mctoil.xlsx", encoding='utf-8-sig')
# df.to_csv("blog_url.csv")

## 1.2 유저리뷰 크롤링

In [85]:
# "url_list.csv" 불러오기
url_load = pd.read_excel("iherb_mctoil.xlsx")
url_load = url_load.drop("Unnamed: 0", axis=1)  # 불필요한 칼럼 삭제

url = url_load['url'][0]
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)

driver.get(url)   # 글 띄우기
time.sleep(1)


In [ ]:
dict = {}    # 전체 크롤링 데이터를 담을 그릇

# 수집할 글 갯수 정하기
number = 30

# 수집한 url 돌면서 데이터 수집
for i in tqdm_notebook(range(0, number)):
    # 글 띄우기
    url = url_list[i]
    chrome_path = chromedriver_autoinstaller.install()
    driver = webdriver.Chrome(chrome_path)

    driver.get(url)   # 글 띄우기
    time.sleep(2)
    
    #리뷰창 들어가기
    review =driver.find_element_by_key("모든 구매후기 보기")
    review.click()
    
    # 크롤링
    
    try : 
        
        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성


        # 리뷰 크롤링
        user id = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents

        content_list = []
        for content in contents:
            content_list.append(content.text)
 
        content_str = ' '.join(content_list)                         # content_str

        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기ii
    if i == 30 or 50 or 80:
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("blog_content.xlsx", encoding='utf-8-sig')
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

# 2. 데이터 전처리

# 3. 추천시스템

# 4. 평가/검증 